# [모듈 8.5] 전체 모델 빌딩 파이프라인

## 0. 기본 세이지 메이커 정보 및 기본 변수 로딩

In [1]:
import boto3
import sagemaker
import pandas as pd

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()

%store -r 
%store

Stored variables and their in-db values:
base_preproc_input_dir                 -> 'opt/ml/processing/input'
claims_data_uri                        -> 's3://sagemaker-ap-northeast-2-057716757052/fraud2
customers_data_uri                     -> 's3://sagemaker-ap-northeast-2-057716757052/fraud2
dataset_path                           -> 'opt/ml/processing/input/dataset.csv'
default_bucket                         -> 'sagemaker-ap-northeast-2-057716757052'
hyperparameters                        -> {'max_depth': '3', 'eta': '0.2', 'objective': 'bin
image_uri                              -> '366743142698.dkr.ecr.ap-northeast-2.amazonaws.com
input_data_uri                         -> 's3://sagemaker-ap-northeast-2-057716757052/fraud2
local_test_path                        -> 'preproc/test/test.csv'
local_train_path                       -> 'preproc/train/train.csv'
pipeline_endpoint_name                 -> 'all-pipeline-endpoint-0414'
preprocessing_code                     -> 'src/preprocessin

## 1. 모델 빌딩 파이프라인 변수 생성



In [2]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType",
    default_value="ml.m5.xlarge"
)

training_instance_type = ParameterString(
    name="TrainingInstanceType",
    default_value="ml.m5.xlarge"
)



input_data = ParameterString(
    name="InputData",
    default_value=input_data_uri,
)


## 2.전처리 스텝 단계 정의

In [3]:
from sagemaker.sklearn.processing import SKLearnProcessor

framework_version = "0.23-1"

sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    base_job_name="sklearn-fraud-process",
    role=role,
)
print("input_data: \n", input_data)

input_data: 
 s3://sagemaker-ap-northeast-2-057716757052/fraud2scratch/input


In [4]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep
    
step_process = ProcessingStep(
    name="FraudScratchProcess",
    processor=sklearn_processor,
    inputs=[
        ProcessingInput(source=input_data_uri,destination='/opt/ml/processing/input'),
         ],
    outputs=[ProcessingOutput(output_name="train",
                              source='/opt/ml/processing/output/train'),
             ProcessingOutput(output_name="test",
                              source='/opt/ml/processing/output/test')],
    code= preprocessing_code_dir,
)


## 3.모델 학습을 위한 학습단계 정의 



In [5]:
# train_preproc_dir_artifact_path = train_preproc_dir_artifact + '/train.csv'

# print(train_preproc_dir_artifact_path)

# train_df = pd.read_csv(train_preproc_dir_artifact_path, header=None)
# # print(train_df.head())

# def get_pos_scale_weight(df, label):
#     '''
#     1, 0 의 레이블 분포를 계산하여 클래스 가중치 리턴
#     예: 1: 10, 0: 90 이면 90/10 = 9 를 제공함. 
#     '''
#     fraud_sum = df[df[label] == 1].shape[0]
#     non_fraud_sum = df[df[label] == 0].shape[0]
#     class_weight = int(non_fraud_sum / fraud_sum)
#     print(f"fraud_sum: {fraud_sum} , non_fraud_sum: {non_fraud_sum}, class_weight: {class_weight}")
#     return class_weight

# class_weight = get_pos_scale_weight(train_df, label=0)


In [6]:
# from sagemaker.estimator import Estimator

# model_prefix = 'fraud2scratch/model'
# model_path = f"s3://{default_bucket}/{model_prefix}"
# image_uri = sagemaker.image_uris.retrieve(
#     framework="xgboost",
#     region=region,
#     version="1.0-1",
#     py_version="py3",
# )
# xgb_train = Estimator(
#     image_uri=image_uri,
#     instance_type=training_instance_type,
#     instance_count=1,
#     output_path=model_path,
#     role=role,
# )
# xgb_train.set_hyperparameters(
#     objective="binary:logistic",    
#     scale_pos_weight = class_weight,
#     num_round=50,
#     max_depth=5,
#     eta=0.2,
#     gamma=4,
#     min_child_weight=6,
#     subsample=0.7,
#     silent=0
# )

### 기본 훈련 변수 및 하이퍼파라미터 설정

In [7]:
from sagemaker.xgboost.estimator import XGBoost

bucket = sagemaker_session.default_bucket()
prefix = 'fraud2train'

estimator_output_path = f's3://{bucket}/{prefix}/training_jobs'
train_instance_count = 1

hyperparameters = {
        "max_depth": "3",
        "eta": "0.2",
        "objective": "binary:logistic",
        "num_round": "100",
}

In [8]:
xgb_train = XGBoost(
    entry_point = "xgboost_starter_script.py",
    source_dir = "src",
    output_path = estimator_output_path,
    code_location = estimator_output_path,
    hyperparameters = hyperparameters,
    role = role,
    instance_count = train_instance_count,
    instance_type = training_instance_type,
    framework_version = "1.0-1")

In [9]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep


step_train = TrainingStep(
    name="FraudScratchTrain",
    estimator=xgb_train,
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            # s3_data= train_preproc_dir_artifact,            
            content_type="text/csv"
        ),
    },
)

'FailureReason': 'ClientError: Key [train] on Property [ProcessingOutputConfig.Outputs] does not exist.',

## 4.모델 평가단계 정의 


In [10]:
# from sagemaker.processing import ScriptProcessor


# script_eval = ScriptProcessor(
#     image_uri=image_uri,
#     command=["python3"],
#     instance_type=processing_instance_type,
#     instance_count=1,
#     base_job_name="script-fraud2scratch-eval",
#     role=role,
# )

In [11]:
# from sagemaker.workflow.properties import PropertyFile


# # evaluation_report = PropertyFile(
# #     name="EvaluationReport",
# #     output_name="evaluation",
# #     path="evaluation.json"
# # )
# step_eval = ProcessingStep(
#     name="FraudEval",
#     processor=script_eval,
#     inputs=[
#         ProcessingInput(
#             source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
#             destination="/opt/ml/processing/model"
#         ),
#         ProcessingInput(
#             source=step_process.properties.ProcessingOutputConfig.Outputs[
#                 "test"
#             ].S3Output.S3Uri,
#             destination="/opt/ml/processing/test"
#         )
#     ],
#     outputs=[
#         ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
#     ],
#     code="src/evaluation.py",
# #     property_files=[evaluation_report],
# )

## 5.세이지 메이커 모델 스텝 생성

In [12]:
from sagemaker.model import Model
    
model = Model(
    image_uri= step_train.properties.AlgorithmSpecification.TrainingImage,
    model_data= step_train.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=sagemaker_session,
    role=role,
)

In [13]:
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.steps import CreateModelStep


inputs = CreateModelInput(
    instance_type="ml.m5.large",
    # accelerator_type="ml.eia1.medium",
)
step_create_model = CreateModelStep(
    name="FraudScratchModel",
    model=model,
    inputs=inputs,
)

## 6.실시간 엔드 포인트 배포 스텝 생성

In [14]:

local_deploy_code_path = 'src/deploy_model.py'
s3_deploy_code_path = f"s3://{default_bucket}/{project_prefix}/code"
s3_deploy_code_uri = sagemaker.s3.S3Uploader.upload(
    local_path=local_deploy_code_path, 
    desired_s3_uri=s3_deploy_code_path,
)
print("s3_deploy_code_uri: ", s3_deploy_code_uri)

pipeline_endpoint_name = 'all-pipeline-endpoint-0414'
endpoint_instance_type = "ml.m5.xlarge"

s3_deploy_code_uri:  s3://sagemaker-ap-northeast-2-057716757052/fraud2scratch/code/deploy_model.py


In [15]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.steps import ProcessingStep

deploy_model_processor = SKLearnProcessor(
    framework_version='0.23-1',
    role= role,
    instance_type="ml.t3.medium",
    instance_count=1,
    base_job_name='fraud-scratch-deploy-model',
    sagemaker_session=sagemaker_session)


deploy_step = ProcessingStep(
    name='DeployModel',
    processor=deploy_model_processor,
    job_arguments=[
        "--model_name", step_create_model.properties.ModelName, 
        "--region", region,
        "--endpoint_instance_type", endpoint_instance_type,
        "--endpoint_name", pipeline_endpoint_name
    ],
    code=s3_deploy_code_uri)

## 7.모델 빌딩 파이프라인 정의

In [16]:
from sagemaker.workflow.pipeline import Pipeline

project_prefix = 'FraudScratchProduction'

pipeline_name = project_prefix
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type, 
        processing_instance_count,
        training_instance_type,        
        input_data,
    ],
    steps=[step_process, step_train, step_create_model, deploy_step],
)

In [17]:
import json

definition = json.loads(pipeline.definition())
# definition

### 파이프라인을 SageMaker에 제출하고 실행하기 


In [18]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/fraudscratchproduction',
 'ResponseMetadata': {'RequestId': '7c56a318-3301-424d-ba1e-4901dfbd1ddf',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7c56a318-3301-424d-ba1e-4901dfbd1ddf',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '95',
   'date': 'Sat, 26 Jun 2021 05:29:35 GMT'},
  'RetryAttempts': 0}}

디폴트값을 이용하여 파이프라인을 샐행합니다. 

In [19]:
execution = pipeline.start()

### 파이프라인 운영: 파이프라인 대기 및 실행상태 확인

워크플로우의 실행상황을 살펴봅니다. 

In [20]:
execution.list_steps()

[]

In [21]:
# execution.describe()
execution.wait()

실행이 완료될 때까지 기다립니다.

실행된 단계들을 리스트업합니다. 파이프라인의 단계실행 서비스에 의해 시작되거나 완료된 단계를 보여줍니다.

In [22]:
execution.list_steps()

[{'StepName': 'DeployModel',
  'StartTime': datetime.datetime(2021, 6, 26, 5, 37, 7, 129000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 6, 26, 5, 54, 55, 207000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:processing-job/pipelines-njzy7i5wy4i2-deploymodel-rrg8l0rsdp'}}},
 {'StepName': 'FraudScratchModel',
  'StartTime': datetime.datetime(2021, 6, 26, 5, 37, 5, 675000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 6, 26, 5, 37, 6, 461000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:model/pipelines-njzy7i5wy4i2-fraudscratchmodel-v8nfn4rdb5'}}},
 {'StepName': 'FraudScratchTrain',
  'StartTime': datetime.datetime(2021, 6, 26, 5, 34, 2, 525000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 6, 26, 5, 37, 5, 52000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'TrainingJob

## 변수 저장

In [23]:
%store pipeline_endpoint_name

Stored 'pipeline_endpoint_name' (str)
